<a href="https://colab.research.google.com/github/arthuruan/branch-and-bound/blob/main/branch-and-bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Branch and Bound

### Ler instância de test

In [13]:
class Data:
    def __init__(self, variablesNumb, restrictionsNumb, coefficientsObjectiveFunc, restrictions):
        self.variablesNumb = variablesNumb
        self.restrictionsNumb = restrictionsNumb
        self.coefficientsObjectiveFunc = coefficientsObjectiveFunc
        self.restrictions = restrictions

    def printData(self):
        print(self.variablesNumb)
        print(self.restrictionsNumb)
        print(self.coefficientsObjectiveFunc)
        print(self.restrictions)


def read_file(path):
    file = open(path, "r")
    
    variablesNumb = 0
    restrictionsNumb = 0
    coefficientsObjectiveFunc = []
    restrictions = []

    for indexLine, line in enumerate(file):
        line = line.split()

        # loading number of variables and restrictions
        if indexLine == 0:
            variablesNumb = int(line[0])
            restrictionsNumb = int(line[1])

        # loading coefficients of the objective function
        elif indexLine == 1:
            for coefficient in line:
                coefficientsObjectiveFunc.append(int(coefficient))
    
        # loading restrictions
        else:
            coeficientsRestriction = []
            for restrictionsLine in line:
                coeficientsRestriction.append(int(restrictionsLine))
            restrictions.append(coeficientsRestriction)
        

    return Data(variablesNumb, restrictionsNumb, coefficientsObjectiveFunc, restrictions)

lendo instância de teste

In [24]:
data1 = read_file('instance-4.txt')
data1.printData()

9
9
[9, 7, 10, 7, 9, 6, 8, 4, 9]
[[4, 9, 4, 1, 9, 6, 3, 6, 1, 40], [3, 7, 8, 7, 6, 3, 5, 9, 4, 80], [9, 3, 6, 5, 7, 1, 1, 3, 9, 40], [5, 9, 6, 5, 9, 7, 8, 7, 8, 10], [7, 7, 4, 1, 3, 4, 8, 1, 9, 10], [1, 6, 6, 1, 6, 7, 3, 8, 7, 10], [6, 6, 8, 6, 10, 8, 1, 4, 4, 70], [9, 1, 9, 7, 10, 5, 6, 2, 5, 10], [2, 7, 6, 5, 1, 1, 9, 2, 1, 20]]


### Configuração do pacote MIP

In [15]:
# instalação e importação do pacote mip
!pip install mip
from mip import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# resolve the model
def solve(model):
    status = model.optimize()
    return model

# save the model in lp file and print the content
def save(model, filename):
    model.write(filename) # save the model in a file
    with open(filename, "r") as f: # read and show the content
        print(f.read())

### Closest Index

Função que retorna o index do item que tem o valor mais próximo ao número de refencia recebido(K).

In [17]:
# function to return the index of closest number to 0.5 of the refence number
def closestIndex(lst, K):
    return lst.index(min(lst, key = lambda x: abs(x - K)))

### Criando modelo

In [25]:
model = Model(sense=MAXIMIZE, solver_name=CBC)

variablesRange = range(data1.variablesNumb)
coeficients = data1.coefficientsObjectiveFunc
restrictionRange = range(data1.restrictionsNumb)
restrictions = data1.restrictions

x = [model.add_var(var_type=BINARY, name=f"x_{v}") for v in variablesRange]
model.objective = xsum(coeficients[v] * x[v] for v in variablesRange)

for r in restrictionRange:
    model += xsum(restrictions[r][v] * x[v] for v in variablesRange) <= restrictions[r][-1]

model.relax()
save(model, "model.lp")

\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2 + 6 x_3 + 10 x_4 + 8 x_5 + x_6 + 4 x_7 + 4 x_8 <= 70
constr(7):  9 x_0 + x_1 + 9 x_2 + 7 x_3 + 10 x_4 + 5 x_5 + 6 x_6 + 2 x_7 + 5 x_8 <= 10
constr(8):  2 x_0 + 7 x_1 + 6 x_2 + 5 x_3 + x_4 + x_5 + 9 x_6 + 2 x_7 + x_8 <= 20
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <

### Algoritmo Branch and Bound 

In [26]:
import math

BRANCHING_RULE = 0.5

# branch and bound
def branchAndBound(nodes, model, dualLimit, primalLimit):
    newNodes = []

    for i, nodeItem in enumerate(nodes):
        node: Model = nodeItem.copy()

        try:
            # solving the node(model)
            solve(node)
            dualLimit = min(dualLimit, node.objective_value)
        except:
            # STOP 1 node
            continue


        # mapping the variables
        variableValues = []
        continuousVars = []
        for v in node.vars:
            variableValues.append(v.x)
            if (v.x != 0) and (v.x != 1):
                continuousVars.append(v.x)

        # it means that all the variables are binaries
        if len(continuousVars) == 0:
            if node.objective_value > primalLimit:
                primalLimit = node.objective_value
                model = node
            # STOP 2 node
            continue

        # getting the closest variable index to BRANCH_RULE value
        index = closestIndex(variableValues, BRANCHING_RULE)

        # adding child node A to the next iteration
        nodeA = node.copy()
        nodeA += nodeA.vars[index] == 0
        newNodes.append(nodeA)

        # adding child node B to the next iteration
        nodeB = node.copy()
        nodeB += nodeB.vars[index] == 1
        newNodes.append(nodeB)

    if len(newNodes) != 0: # it means that dose
        childModel, childPrimalLimit = branchAndBound(newNodes, model, dualLimit, primalLimit)

        if childPrimalLimit > primalLimit:
            primalLimit = childPrimalLimit # best primal limit
            model = childModel # best model
    
    return model, primalLimit

### Main

In [27]:
nodes = [model] # initial node (root)

dualLimit = math.inf # upper limit
primalLimit = -math.inf # lower limit

model, primal = branchAndBound(nodes, model, dualLimit, primalLimit)

print(model.objective_value)
print("Solution:")
for v in model.vars:
    print(f"{v.name} = {v.x:.2f}")

10.0
Solution:
x_0 = 0.00
x_1 = 0.00
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.00
